In [37]:
from utils.get_data import get_user, get_project, get_all_projects, get_all_users
from tqdm import tqdm

In [16]:
embedds_file = open('/Users/andrey.krotkikh/Downloads/wiki.ru.vec', 'r')
lines = embedds_file.readlines()

In [24]:
len(lines)

1888424

In [27]:
import fasttext
import fasttext.util

ft = fasttext.load_model('cc.ru.300.bin')
ft.get_dimension()
fasttext.util.reduce_model(ft, 50)
ft.get_dimension()

ValueError: cc.ru.300.bin cannot be opened for loading!

In [31]:
import fasttext.util
fasttext.util.download_model('ru', if_exists='ignore')  # English
ft = fasttext.load_model('cc.ru.300.bin')

In [32]:
ft.get_dimension()

300

In [33]:
fasttext.util.reduce_model(ft, 50)
ft.get_dimension()

50

In [34]:
ft.words

[',',
 '.',
 'и',
 'в',
 '</s>',
 ':',
 ')',
 '(',
 'на',
 '"',
 'с',
 'не',
 '»',
 'для',
 '-',
 '«',
 '/',
 'по',
 '—',
 'что',
 'В',
 '!',
 'из',
 'от',
 'к',
 'как',
 '?',
 'а',
 '–',
 'за',
 ';',
 'о',
 'или',
 'это',
 '�',
 '1',
 '...',
 '_',
 "'",
 'его',
 'у',
 'до',
 '|',
 '2',
 'но',
 'все',
 'года',
 '+',
 'я',
 'то',
 '%',
 'при',
 'он',
 'так',
 'же',
 'только',
 '3',
 'их',
 'А',
 'можно',
 '#',
 'был',
 'И',
 'время',
 'также',
 'году',
 '10',
 'было',
 'будет',
 'может',
 'вы',
 '0',
 'уже',
 '[',
 ']',
 '4',
 '>',
 'чтобы',
 'есть',
 'На',
 'если',
 '5',
 '…',
 'г',
 'которые',
 'С',
 'без',
 'со',
 'очень',
 'бы',
 'мы',
 'этом',
 'более',
 'под',
 'они',
 'быть',
 'России',
 'во',
 'Я',
 'после',
 'еще',
 '№',
 '7',
 'того',
 '20',
 'Но',
 'том',
 'лет',
 '12',
 'она',
 'когда',
 'этого',
 'ее',
 'который',
 '6',
 '15',
 'Если',
 'UTC',
 'Вы',
 'Это',
 '*',
 'нет',
 'были',
 'Как',
 'области',
 'работы',
 'меня',
 'была',
 '8',
 'где',
 'По',
 'чем',
 '\xad',
 'Для',

In [36]:
ft.get_word_vector('день')

array([-0.20272045,  0.09386399,  0.04327361, -0.04959331,  0.00915238,
       -0.08465039, -0.10279   ,  0.19168155, -0.16890833,  0.12588993,
       -0.048469  , -0.06678698, -0.12838818, -0.04901456, -0.08931329,
       -0.03507445, -0.22597477,  0.05564087, -0.01911009, -0.12128128,
       -0.2035756 ,  0.12580894,  0.02827996,  0.15265557,  0.0327943 ,
       -0.0970552 , -0.10492814,  0.0742178 ,  0.08705788,  0.09618815,
        0.1067735 , -0.15651417,  0.0596566 ,  0.05934925,  0.07334846,
        0.02816094, -0.14086004, -0.00444515,  0.02002926, -0.04343377,
       -0.089909  , -0.0152721 ,  0.06721427,  0.3300016 ,  0.18010509,
        0.1546408 , -0.11751419,  0.14518057, -0.03955141, -0.00901164],
      dtype=float32)

In [35]:
dict_of_embeds = {}

In [38]:
for word in tqdm(ft.words):
    dict_of_embeds[word] = ft.get_word_vector(word)

100%|██████████| 2000000/2000000 [00:27<00:00, 72301.32it/s]


In [ ]:
dict_of_embeds

In [39]:
import pickle

In [40]:
with open('embedds.pickle', 'wb') as handle:
    pickle.dump(dict_of_embeds, handle, protocol=pickle.HIGHEST_PROTOCOL)